In [1]:
import pandas as pd

import pathlib as pl

import os

In [19]:
import shutil

In [1]:
data_folder = pl.Path("data")

labels_df = pd.read_csv(data_folder / "train.csv")

In [2]:
labels_df.head()

,ImageId,ClassId,EncodedPixels
0,0002cc93b.jpg,1,29102 12 29346 24 29602 24 29858 24 30114 24 3...
1,0007a71bf.jpg,3,18661 28 18863 82 19091 110 19347 110 19603 11...
2,000a4bcdd.jpg,1,37607 3 37858 8 38108 14 38359 20 38610 25 388...
3,000f6bf48.jpg,4,131973 1 132228 4 132483 6 132738 8 132993 11 ...
4,0014fce06.jpg,3,229501 11 229741 33 229981 55 230221 77 230468...


In [3]:
len(labels_df)

7095

In [4]:
labels_df.ClassId.value_counts()

3    5150
1     897
4     801
2     247
Name: ClassId, dtype: int64

In [10]:
labels_df.ImageId.value_counts()

db4867ee8.jpg    3
ef24da2ba.jpg    3
940e8cd2f.jpg    2
39c094b3d.jpg    2
0cb590f8e.jpg    2
                ..
270f5361d.jpg    1
517f3ce34.jpg    1
02d8c6b30.jpg    1
a922818e9.jpg    1
395381ac8.jpg    1
Name: ImageId, Length: 6666, dtype: int64

In [21]:
train_image_folder = data_folder / "train_images"
test_image_folder = data_folder / "test_images"

In [5]:
train_image_files = os.listdir(train_image_folder)

In [7]:
len(train_image_files)

12568

In [8]:
test_image_files = os.listdir(test_image_folder)

In [9]:
len(test_image_files)

5507

In [11]:
severstal_binary_folder = data_folder / "severstal-binary"

In [24]:
shutil.rmtree(severstal_binary_folder)

In [12]:
os.makedirs(severstal_binary_folder / "train" / "good")
os.makedirs(severstal_binary_folder / "train" / "bad")

os.makedirs(severstal_binary_folder / "test" / "good")
os.makedirs(severstal_binary_folder / "test" / "bad")

In [23]:

bad_images = labels_df.ImageId.values

for image in train_image_files:
    if image in bad_images:
        shutil.copy(train_image_folder / image, severstal_binary_folder / "train" / "bad" /image)
    else:
        shutil.copy(train_image_folder / image, severstal_binary_folder / "train" / "good" /image)
        